# Discounted Cash Flow (DCF) Valuation – Unilever (ULVR.L)

## Project Overview
This project builds an end-to-end Discounted Cash Flow (DCF) valuation model in Python using Jupyter Notebook to estimate the intrinsic (fair) value of Unilever PLC.

The analysis begins with historical financial statement data to understand revenue
trends, operating performance, and profitability. Based on this analysis, revenue
is forecasted using conservative growth assumptions, followed by an **EBIT forecast** derived from normalized operating margins.

To reflect operating performance after taxes, **Net Operating Profit After Tax (NOPAT)**is calculated and used as a core input for estimating future **Free Cash Flow to the Firm (FCFF)**.

The projected cash flows are discounted using a **Weighted Average Cost of Capital (WACC)**, estimated through the CAPM framework for cost of equity and market-based assumptions for the cost of debt. A terminal value is then calculated to capture long-term cash flow generation beyond the explicit forecast period.

The final output of the model is an estimate of **Enterprise Value and Equity Value**, which is compared against the current market price to assess potential upside or downside and support an investment view.



### Table of Contents

1. [Importing Libraries and Data](#importing-libraries-and-data)
2. [Cleaning Data](#cleaning-data)
3. [Revenue Forecast](#revenue-forecast)
4. [Operating Margin and EBIT Forecast](#operating-margin-and-ebit-forecast)
5. [NOPAT Forecast](#nopat-forecast)
6. [CAPM, Cost of Debt, Cost of Equity](#capm-cost-of-debt-cost-of-equity)
7. [WACC Estimation](#wacc-estimation)
8. [FCFF Growth Rate and FCFF Forecast](#fcff-growth-rate-and-fcff-forecast)
9. [Discounting Cash Flows](#discounting-cash-flows)
10. [Terminal Growth](#terminal-growth)
11. [Discounting Terminal Value](#discounting-terminal-value)
12. [Enterprise Value and Equity Value](#enterprise-value-and-equity-value)
13. [Conclusion](#investment-insight-and-conclusion)



#### Importing Libraries and Data 

In [2]:
import pandas as pd
import yfinance as yf

ul = yf.Ticker('ULVR.L')

print(ul.info['longName']) 

df_pl = ul.financials
df_balancesheet = ul.balancesheet
df_cashflow  = ul.cashflow 

Unilever PLC


#### Data Cleaning and Preparation

In [3]:
pd.options.display.float_format = '{:,.1f}'.format
df_scaled = df_pl / 1_000_000
df_scaled = df_balancesheet / 1_000_000
df_scaled = df_cashflow / 1_000_000

df_pl = df_pl.T
df_cashflow = df_cashflow.T
df_balancesheet = df_balancesheet.T


df_pl.columns = df_pl.columns.str.strip().str.lower().str.replace(" ","_")
df_balancesheet.columns = df_balancesheet.columns.str.strip().str.lower().str.replace(" ","_")
df_cashflow.columns = df_cashflow.columns.str.strip().str.lower().str.replace(" ","_")


df_pl = df_pl.sort_index(ascending = True)
df_balancesheet = df_balancesheet.sort_index(ascending = True)
df_cashflow = df_cashflow.sort_index(ascending = True)

df_pl.index = df_pl.index.year
df_cashflow.index = df_cashflow.index.year
df_balancesheet.index = df_balancesheet.index.year

df_pl.index = df_pl.index.astype(float)
df_balancesheet.index = df_balancesheet.index.astype(float)

#### Revenue Forecast

In [4]:
# Average Revenue Growth (YOY)
revenue = df_pl['total_revenue']
revneue_growth = 0.025     # Conservative growth assumption

# Forecasting Revenue for 5 years (Loop)
last_revenue = revenue.iloc[-1]   
revenue_growth = 0.025      
forecast_years = 5
last_year = 2024

revenue_forecast = {}

for i in range(1,forecast_years +1):
    year = last_year + i
    last_revenue = last_revenue*(1+revenue_growth)
    revenue_forecast[year] = last_revenue.round(2)

revenue_forecast

{2025: np.float64(62280025000.0),
 2026: np.float64(63837025625.0),
 2027: np.float64(65432951265.62),
 2028: np.float64(67068775047.27),
 2029: np.float64(68745494423.45)}

#### Operating Margin (EBIT Margin) & Ebit Forecast

In [5]:
# Historical EBIT margin
ebit = df_pl['ebit']

ebit_margin = ebit/revenue
avg_ebitmargin = ebit_margin.mean()

# EBIT forecast 5 years (Loop)
ebit_forecast = {}

for year, last_revenue in revenue_forecast.items():
    ebit_forecast[year] = (last_revenue *avg_ebitmargin).round(2)
ebit_forecast

{2025: np.float64(10851273089.64),
 2026: np.float64(11122554916.88),
 2027: np.float64(11400618789.8),
 2028: np.float64(11685634259.55),
 2029: np.float64(11977775116.04)}

#### NOPAT Forecast

In [6]:
# NOPAT Forecast ( EBIT after Tax )
tax_rate = 0.25   # Normalized effective tax rate
nopat_forecast = {}

for year, ebit_value in ebit_forecast.items():
    nopat_forecast[year] = round(ebit_value * (1 - tax_rate), 2)

nopat_forecast

{2025: np.float64(8138454817.23),
 2026: np.float64(8341916187.66),
 2027: np.float64(8550464092.35),
 2028: np.float64(8764225694.66),
 2029: np.float64(8983331337.03)}

#### Cost of Capital (CAPM, Cost of Debt, Cost of Equity)

In [7]:

# Calculation Parameters for (Cost of debt, cost of equity, WACC)
tax_rate = 0.25 #---------------------for cost of debt
risk_free_rate = 0.04 #---------------for cost of equity
equity_risk_pre = 0.055 #-------------for cost of equity
beta = 0.70 #-------------------------for cost of equity

# Cost of Equity (Calculation)
cost_of_equity = risk_free_rate + beta * equity_risk_pre

In [8]:
interest_ex_mean = df_pl['interest_expense'].mean() #------------for cost of debt
debt_mean = df_balancesheet['total_debt'].mean() #--------for cost of debt
market_cap = ul.fast_info['marketCap'] #Equity

# Cost of Debt (Calculation)
cod = interest_ex_mean/debt_mean 
after_tax_cod = cod * (1 - tax_rate)

### WACC Estimation

In [9]:
#Capital Structure
equity  = market_cap
total_capital = equity + debt_mean

weighted_equity = equity/total_capital
weighted_debt = debt_mean/total_capital

# WACC Calculation
wacc = weighted_equity *cost_of_equity + weighted_debt *after_tax_cod
wacc.round(2)

np.float64(0.08)

#### FCFF Growth Rate & FCFF Forecast

In [10]:
# FCFF growth rate calculation 
fcff = df_cashflow['free_cash_flow']
fcff_growth = fcff.pct_change()
fcff_growth.mean()*100

#------------------------------------------|

fcff_last = float(fcff.loc[2024])   # FORCE scalar
fcff_growth = 0.0671                # your calculated growth
forecast_years = 5
last_year = 2024

fcff_forecast = {}

for i in range(1, forecast_years + 1):
    year = last_year + i
    fcff_last = fcff_last * (1 + fcff_growth)
    fcff_forecast[year] = round(fcff_last, 2)

fcff_forecast

{2025: 8054470800.0,
 2026: 8594925790.68,
 2027: 9171645311.23,
 2028: 9787062711.62,
 2029: 10443774619.57}

#### Discounting Cash Flow

In [11]:
# Discounting the FCFF to current time period (5 years)
last_year = 2024
wacc = 0.08

discounted_fcff = {}

for year, fcff_value in fcff_forecast.items():
    t = year - last_year
    discounted_fcff[year] = round(
        fcff_value / ((1 + wacc) ** t), 2
    )

discounted_fcff

{2025: 7457843333.33,
 2026: 7368763537.96,
 2027: 7280747751.26,
 2028: 7193783264.23,
 2029: 7107857519.69}

#### Terminal growth

In [12]:
# Terminal Growth
terminal_growth = 0.02

fcff_2029 = fcff_forecast[2029]

terminal_value_2029 = (
    fcff_2029 * (1 + terminal_growth)
) / (wacc - terminal_growth)

terminal_value_2029

177544168532.69

#### Discounting Terminal Value

In [13]:
# Discounting the terminal value to current time period (Beyond 5 years)

l = 5

discounted_terminal_value = terminal_value_2029 / ((1 + wacc) ** l)

round(discounted_terminal_value, 2)

120833577834.68

In [14]:
# Enterprise Value
# – Net Debt
# = Equity Value
# ÷ Shares
# = Intrinsic Price
# vs Market Price
# → BUY / HOLD / SELL


### Enterprise Value and Equity Value

In [15]:
# Extracting Current Price and Shares Outstanding
net_debt = df_balancesheet['net_debt']
current_price = ul.info['currentPrice']/100       # converting from pence to pounds
shares_outstanding = ul.info['sharesOutstanding'] # shares_outstanding ≈ 2_744_000_000

#-----------------------------------------------------------------
# Enterprise Value -> Equity Value -> Fair value
enterprise_value = sum(discounted_fcff.values()) + discounted_terminal_value  # Sum of discounted fcffs andd disounted terminal value
equity_value = enterprise_value - net_debt.iloc[-1]
intrinsic_value  = equity_value/shares_outstanding
upside_pct = round((intrinsic_value- current_price) / current_price * 100, 2)

### Investment Insight and Conclusion

Based on the Discounted Cash Flow (DCF) valuation, Unilever PLC’s intrinsic value
is estimated to be 61.54, which is **above the current market price** of 48.64. This suggests that the stock is **undervalued**, offering an upside potential of **26.53%.**


**Conclusion:** Given the estimated upside and the stability of cash flows,
the stock presents a **moderate upside potential** under base-case assumptions.

In [16]:
apple_forecast = pd.DataFrame({
    'Revenue':revenue,
    'Revenue Forecast': revenue_forecast,
    'Operating Margin Forecast':ebit_forecast,
    'NOPAT Forecast':nopat_forecast,
    'FCFF Forecast': fcff_forecast,
    'Discounted FCFF': discounted_fcff,
    'Terminal Value': terminal_value_2029,
    'Discounted Terminal Value': discounted_terminal_value
})

apple_forecast.to_csv('apple_forecast.csv')

In [22]:
intrinsic_value

np.float64(61.54453527265603)